In [1]:
import os
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr #Loocv

In [2]:
import sys
sys.path.append('../Clustering/custom_function/')
import clustering

# Categorical model

In [3]:
regions = ['USA','UK','South_africa','Australia','Germany','Japan',
            'CHN','HK','France','Chile','Mexico','Spain','Portugal',
            'Brazil','Israel','Russia','Egypt','Qatar','India']

In [4]:
# contain raw output of models
for num in range(40,71):
    os.makedirs('output_data/categorical_model/'+str(num))
    for region in regions:
        raw_file = 'output_data/full_feature/'+str(num)+'/'+region+'.csv'
        raw_df = pd.read_csv(raw_file,index_col=0)
        # clustering: HPP
        raw_results = clustering.create_cluster_models(data_type = "df",data = raw_df,
                            methods=['KMeans'],ks=[3],keep_orig=True)
        raw_results.to_csv('output_data/categorical_model/'+str(num)+'/'+region+'_raw_3k.csv')

In [5]:
# contain raw output of models
for num in [i for i in range(5,40)]+[i for i in range(71,159)]:
    os.makedirs('output_data/categorical_model/'+str(num))
    for region in regions:
        raw_file = 'output_data/full_feature/'+str(num)+'/'+region+'.csv'
        raw_df = pd.read_csv(raw_file,index_col=0)
        # clustering: HPP
        raw_results = clustering.create_cluster_models(data_type = "df",data = raw_df,
                            methods=['KMeans'],ks=[3],keep_orig=True)
        raw_results.to_csv('output_data/categorical_model/'+str(num)+'/'+region+'_raw_3k.csv')

# RDM: 0/1 matrix

In [6]:
def cluster_rdm(cluster_results_region,cluster_model='cl_k3'):
    dissim_cluster_result = pd.DataFrame(index = cluster_results_region.index,
                                         columns = cluster_results_region.index)
    for i in dissim_cluster_result.index:
        for j in dissim_cluster_result.columns:
            # cluster result was in the last column
            if cluster_results_region.loc[i,cluster_model] == cluster_results_region.loc[j,cluster_model]:
                dissim_cluster_result.loc[i,j] = 0
            else:
                dissim_cluster_result.loc[i,j] = 1
            dissim_cluster_result = dissim_cluster_result.astype(float)
            
    return(dissim_cluster_result)

In [7]:
subset_rdm_nums = {}
for num in range(5,159):
    os.makedirs('output_data/categorical_rdm/'+str(num))
    cluster_raw_dissim_paths = glob.glob('output_data/categorical_model/'+str(num)+'/*.csv')
    subset_rdm = {}
    # kmeans clustering using raw data
    for region_dir in cluster_raw_dissim_paths:
        region_input = pd.read_csv(region_dir, index_col=0)
        region_cluster_rdm = cluster_rdm(region_input,cluster_model='cl_k3')

        region_csv = region_dir.split('\\')[1]
        region_cluster_rdm.to_csv('output_data/categorical_rdm/'+str(num)+'/'+region_csv)

        region = region_dir.split('\\')[1].split('_raw')[0]
        subset_rdm[region] = region_cluster_rdm
        
    subset_rdm_nums[num] =  subset_rdm